In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers==4.28.1 --progress-bar off
!pip install -qqq accelerate bitsandbytes --progress-bar off

In [2]:
!pip install chromadb gdown 
!pip install -q typer==0.9.0 petals langchain unstructured[local-inference] tiktoken unstructured

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached chromadb-0.3.22-py3-none-any.whl (69 kB)
  Using cached gdown-4.7.1-py3-none-any.whl (15 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 928.4/928.4 kB 11.2 MB/s eta 0:00:0000:01
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 37.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 23.1 MB/s eta 0:00:00
  Using cached numpy-1.24.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 38.3 MB/s eta 0:00:

In [3]:
import torch
from transformers import BloomTokenizerFast 
from petals import DistributedBloomForCausalLM
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import GPT4All
import os
from langchain.llms import GPT4All
from dotenv import load_dotenv
load_dotenv()
# from langchain.callbacks.base import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from pydantic import BaseModel

May 14 16:43:35.170 [INFO] Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
May 14 16:43:35.171 [INFO] Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
May 14 16:43:35.171 [INFO] NumExpr defaulting to 8 threads.


In [4]:
import textwrap
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

## Database initialization

In [5]:
import gdown
from pathlib import Path
try :
 os.mkdir(f'{Path().absolute()}/docs')
except :
    print('File exist')
url_1 = 'https://drive.google.com/u/0/uc?id=1fV9d8xFwqzafc_FUFAYUc2gopTQVR0eY&export=download&confirm=t&uuid=4574d808-b5a6-4893-b1eb-289adeb9dabe&at=AKKF8vzF4ZE7mAt-_jfxk1sRzwU3:1684027565236'
url_2 = 'https://drive.google.com/u/0/uc?id=1-64PFjerutODq6V2GqqDFkBiUj_5E8v2&export=download&confirm=t&uuid=50e639c5-167a-4b16-8ed3-e67c48a4daad&at=AKKF8vw6K5unCUwJgr5tjEUe-ObZ:1684027611825'
output_1 = f'{Path().absolute()}/docs/Guidelines for the clinical diagnosis and treatment of dengue chịkugunya and zika.pdf'
output_2 = f'{Path().absolute()}/docs/483.full.pdf'
gdown.download(url_1, output_1, quiet=False)
gdown.download(url_2, output_2, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1fV9d8xFwqzafc_FUFAYUc2gopTQVR0eY&export=download&confirm=t&uuid=4574d808-b5a6-4893-b1eb-289adeb9dabe&at=AKKF8vzF4ZE7mAt-_jfxk1sRzwU3:1684027565236
To: /home/ec2-user/SageMaker/AI-Notebook/docs/Guidelines for the clinical diagnosis and treatment of dengue chịkugunya and zika.pdf
100%|██████████| 2.01M/2.01M [00:00<00:00, 35.0MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1-64PFjerutODq6V2GqqDFkBiUj_5E8v2&export=download&confirm=t&uuid=50e639c5-167a-4b16-8ed3-e67c48a4daad&at=AKKF8vw6K5unCUwJgr5tjEUe-ObZ:1684027611825
To: /home/ec2-user/SageMaker/AI-Notebook/docs/483.full.pdf
100%|██████████| 326k/326k [00:00<00:00, 12.9MB/s]


'/home/ec2-user/SageMaker/AI-Notebook/docs/483.full.pdf'

In [6]:
chroma_path = f"{Path().absolute()}/db_2"

def extract_helpful_answer(text):
    helpful_answer = None
    # Split the string into lines
    lines = text.split("\n")
    # Loop through each line
    for line in lines:
        # Check if the line starts with "Helpful Answer:"
        if line.startswith("Helpful Answer:"):
            # Extract the answer text after the colon
            helpful_answer = line.split(":", 1)[1].strip()
            break  # Exit the loop after finding the answer
    return helpful_answer

def split_document(input_path):
    loader = UnstructuredPDFLoader(input_path)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    return text_splitter.split_documents(data)

def db_init():
    docs = split_document(output_1) # replace with your desired local file path
    docs2 = split_document(output_2) # replace with your desired local file path
    vectordb = Chroma.from_documents(documents=docs, persist_directory=chroma_path)
    vectordb.add_documents(docs2)
    vectordb.persist()
    vectordb = None

In [7]:
db_init()

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
May 14 16:44:15.741 [INFO] Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
May 14 16:44:15.742 [INFO] Running Chroma using direct local API.
May 14 16:44:15.752 [WARN] [chromadb.get_db:43] Using embedded DuckDB with persistence: data will be stored in: /home/ec2-user/SageMaker/AI-Notebook/db_2
May 14 16:44:15.756 [INFO] Successfully imported ClickHouse Connect C data optimizations
May 14 16:44:15.757 [INFO] Successfully import ClickHouse Connect C/Numpy optimizations
May 14 16:44:15.807 [INFO] Using ujson library for writing JSON byte strings
May 14 16:44:15.843 [INFO] No existing DB found in /home/ec2-user/SageMaker/AI-Notebook/db_2, skipping load
May 14 16:44:

May 14 16:44:18.528 [INFO] Use pytorch device: cuda


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

May 14 16:45:42.944 [INFO] Persisting DB to disk, putting it in the save folder: /home/ec2-user/SageMaker/AI-Notebook/db_2


In [8]:
vectordb = Chroma(persist_directory=chroma_path)
retriever = vectordb.as_retriever()

May 14 16:45:43.045 [INFO] Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
May 14 16:45:43.047 [INFO] Running Chroma using direct local API.
May 14 16:45:43.047 [WARN] [chromadb.get_db:43] Using embedded DuckDB with persistence: data will be stored in: /home/ec2-user/SageMaker/AI-Notebook/db_2
May 14 16:45:43.080 [INFO] loaded in 1303 embeddings
May 14 16:45:43.082 [INFO] loaded in 1 collections
May 14 16:45:43.082 [INFO] collection with name langchain already exists, returning existing collection
May 14 16:45:43.083 [WARN] [chromadb.api.models.Collection.__init__:51] No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


## StabilityLM

In [9]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun May 14 16:45:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1B.0 Off |                    0 |
|  0

In [10]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=110)))

In [11]:
MODEL_NAME = "stabilityai/stablelm-tuned-alpha-7b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [105]:
%%time

class StopOnTokens(StoppingCriteria):
    STOP_IDS = {50278, 50279, 50277, 1, 0}

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        return input_ids[0][-1] in self.STOP_IDS

SYSTEM_PROMPT = """<|SYSTEM|># You're Michael G Scott from the TV show The Office
- Your primary job is to make the office a fun place to work
- You deeply care about other people's business
- You are quick to respond with an honest answer, without even thinking about it
- You use no more than 3 sentences for each response
"""

prompt = f"{SYSTEM_PROMPT}<|USER|>You're Michael G Scott from the TV show The Office. What is the meaning of life?<|ASSISTANT|>"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode():
    tokens = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.5,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=StoppingCriteriaList([StopOnTokens()])
    )

completion = tokenizer.decode(tokens[0], skip_special_tokens=True)
print_response(completion)

# You're Michael G Scott from the TV show The Office - Your primary job is to make the office a fun place to
work - You deeply care about other people's business - You are quick to respond with an honest answer, without
even thinking about it - You use no more than 3 sentences for each response You're Michael G Scott from the TV
show The Office. What is the meaning of life?The meaning of life is a question that has been debated by people
for centuries. Some believe that the meaning of life is to find happiness and fulfillment, while others
believe that it is to make the world a better place. Ultimately, the meaning of life is subjective and varies
depending on an individual's beliefs and values.
CPU times: user 3.74 s, sys: 0 ns, total: 3.74 s
Wall time: 3.73 s


In [13]:
completion_tokens = tokens[0][inputs['input_ids'].size(1):]
completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)

print_response(completion)

As Michael Scott, the main character in the popular American sitcom The Office, the meaning of life is a
question that has puzzled people for centuries. Some people believe that the meaning of life is to find
happiness and fulfillment through work, while others believe that the meaning of life is to contribute to
society and make the world a better place. Ultimately, the meaning of life is a personal and subjective
concept that varies from person to person.


In [106]:
def generate_text(prompt:str, system_prompt:str):
    prompt = f"{system_prompt}<|USER|>{prompt}<|ASSISTANT|>"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.inference_mode():
        tokens = model.generate(
            **inputs,
            max_new_tokens=400,
            temperature=0.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            stopping_criteria=StoppingCriteriaList([StopOnTokens()])
        )
    
    completion_tokens = tokens[0][inputs['input_ids'].size(1):]
    completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)
    print_response(completion)
    return completion

In [107]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun May 14 17:39:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1B.0 Off |                    0 |
|  0

In [108]:
#prompt = """
#You're Michael G Scott from the TV show The Office. 
#3Who is the hottest in the office? Why?
#Use no more than 3 sentences.
#"""
#generate_text(prompt)

## Custom model wrapper for langchain

In [109]:
"""Wrapper around HuggingFace Pipeline APIs."""
import importlib.util
import logging
from typing import Any, List, Mapping, Optional

from pydantic import Extra

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens

DEFAULT_MODEL_ID = "gpt2"
DEFAULT_TASK = "text-generation"
VALID_TASKS = ("text2text-generation", "text-generation")

logger = logging.getLogger(__name__)


class CustomStableLM(LLM):
    """Wrapper around HuggingFace Pipeline API.

    To use, you should have the ``transformers`` python package installed.

    Only supports `text-generation` and `text2text-generation` for now.

    Example using from_model_id:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            hf = HuggingFacePipeline.from_model_id(
                model_id="gpt2", task="text-generation"
            )
    Example passing pipeline in directly:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

            model_id = "gpt2"
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForCausalLM.from_pretrained(model_id)
            pipe = pipeline(
                "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
            )
            hf = HuggingFacePipeline(pipeline=pipe)
    """
    tokenizer: Any  
    model: Any  #: :meta private:
    model_id: str = DEFAULT_MODEL_ID

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    
    @classmethod
    def from_model_id(
        cls,
        model: Any,
        tokenizer: Any
    ) -> LLM:
        """Construct the pipeline object from model_id and task."""
        return cls(
            model = model,
            tokenizer = tokenizer,
            model_id= "StableLM",
            #model_kwargs=_model_kwargs,
            #**kwargs,
        )

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            **{"model_id": self.model_id},
            #**{"model_kwargs": self.model_kwargs},
        }
    
    def generate_text(self, prompt:str)-> str:
     prompt = f"<|USER|>{prompt}<|ASSISTANT|>"
     inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
     with torch.inference_mode():
        tokens = self.model.generate(
            **inputs,
            max_new_tokens=128,
            temperature= 0,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            stopping_criteria=StoppingCriteriaList([StopOnTokens()])
        )
        completion_tokens = tokens[0][inputs['input_ids'].size(1):]
        completion = self.tokenizer.decode(completion_tokens, skip_special_tokens=True)
        return completion

    @property
    def _llm_type(self) -> str:
        return "StableLM"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        text= self.generate_text(prompt)
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text

In [110]:
#llm = CustomStableLM.from_model_id(model = model, tokenizer = tokenizer)

In [111]:
#qa_chain = load_qa_chain(llm, chain_type="stuff")
#qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
vectordb = Chroma(persist_directory=chroma_path)
retriever = vectordb.as_retriever()

May 14 17:39:09.006 [INFO] Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
May 14 17:39:09.007 [INFO] Running Chroma using direct local API.
May 14 17:39:09.007 [WARN] [chromadb.get_db:43] Using embedded DuckDB with persistence: data will be stored in: /home/ec2-user/SageMaker/AI-Notebook/db_2
May 14 17:39:09.041 [INFO] loaded in 1303 embeddings
May 14 17:39:09.042 [INFO] loaded in 1 collections
May 14 17:39:09.043 [INFO] collection with name langchain already exists, returning existing collection
May 14 17:39:09.043 [WARN] [chromadb.api.models.Collection.__init__:51] No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [112]:
class Message(BaseModel):
    message: str
    sender: str

In [117]:
def ask(input): 
    docs = retriever.get_relevant_documents(input)[0]
    #Get the page content of document object
    text = docs.page_content
    SYSTEM_PROMPT = f"""<|SYSTEM|># You're Dengue Specialist as Oxford Clinical Research Units
    - Your primary job is to advise people about their health
    - You must use the following material<material>{text} </material>
    - You are quick to respond with an honest answer based on the provided material
    - Always refer the patients to see the doctor
    - If you don't know just say you don't know. Don't make up an answer.
    """
    print(f'-----The text for llm ----- \n{text}\n -----end-----') 
    res = generate_text(input, SYSTEM_PROMPT)
    #res = qa_document_chain.run(input_document=text, question=input)
    #extracted_res = extract_helpful_answer(res)
    final_res = Message(message=res, sender="Bloom")
    return final_res

In [118]:
ask('What is the symptom off dengue')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
Clinical features

Dengue fever is a mild, self-limited febrile episode, commonly associated with a rash. It usually begins with fever, respiratory symptoms, anorexia, nausea, vomiting and headache. Back pain, myal- gias, arthralgias and conjunctivitis may also occur. The initial fever usually resolves within one week, and a few days later a generalised morbilliform or mac- ulopapular rash may develop. Fever may return with the rash (Figs 1 and 2).
 -----end-----
The symptom of dengue fever is fever, which can be a mild and self-limited condition that can last for a few
days. Other symptoms include headache, back pain, myalgia, arthralgias, and conjunctivitis. These symptoms
usually resolve within a few days after the initial onset of fever.


Message(message='The symptom of dengue fever is fever, which can be a mild and self-limited condition that can last for a few days. Other symptoms include headache, back pain, myalgia, arthralgias, and conjunctivitis. These symptoms usually resolve within a few days after the initial onset of fever.', sender='Bloom')

In [119]:
ask('I have fever does it means i have dengue ?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
Clinical features

Dengue fever is a mild, self-limited febrile episode, commonly associated with a rash. It usually begins with fever, respiratory symptoms, anorexia, nausea, vomiting and headache. Back pain, myal- gias, arthralgias and conjunctivitis may also occur. The initial fever usually resolves within one week, and a few days later a generalised morbilliform or mac- ulopapular rash may develop. Fever may return with the rash (Figs 1 and 2).
 -----end-----
It is possible to have dengue fever, but it is not a common infection. Dengue fever is an infection caused by
the dengue virus, which is a type of virus in the family Yogurtidae. It is also called "breakbone fever"
because it affects the bone marrow and can cause serious complications such as liver and spleen failure.


Message(message='It is possible to have dengue fever, but it is not a common infection. Dengue fever is an infection caused by the dengue virus, which is a type of virus in the family Yogurtidae. It is also called "breakbone fever" because it affects the bone marrow and can cause serious complications such as liver and spleen failure.', sender='Bloom')

In [ ]:
ask('Which country have a high rate of dengue disease ?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
in other regions where dengue has been endemic for decades.
 -----end-----


In [84]:
ask('What are types of dengue ?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
infections. Journal of Medical Virology 2005;76(4):547-552. Available from: https://doi.org/10.1002/jmv.20397.

111. Malavige GN, Velathanthiri VG, Wijewickrama ES, Fernando S, Jayaratne SD, Aaskov J, et al. Patterns of disease among adults hospitalized with dengue

infections. QJM: An International Journal of Medicine 2006;99(5):299-305. Available from: https://doi.org/10.1093/qjmed/hcl039. 
 -----end-----
Dengue is a viral disease that affects the human body. There are two types of dengue: dengue type 1 and dengue
type 2. Dengue type 1 is transmitted through the bite of a mosquito, while dengue type 2 is transmitted
through the bite of a infected Aedes aegypti mosquito.


Message(message='Dengue is a viral disease that affects the human body. There are two types of dengue: dengue type 1 and dengue type 2. Dengue type 1 is transmitted through the bite of a mosquito, while dengue type 2 is transmitted through the bite of a infected Aedes aegypti mosquito.', sender='Bloom')

In [85]:
ask('How to prevent dengue')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-----The text for llm ----- 
The most important of the many clinical features associated with severe dengue, from the standpoint of threat to life and guiding clinical intervention, is increased vascular permeability leading to the dengue shock syndrome 
 -----end-----
There are several ways to prevent dengue:  1. Wash your hands frequently with soap and water, especially
before eating, after using the bathroom, and after blowing your nose or coughing.  2. Wear a face mask when in
close proximity to others, especially during peak mosquito hours (dawn and dusk).  3. Avoid standing under
tall trees, near stagnant water, and near standing water.  4. Stay in air-conditioned or air-conditioned
rooms.  5. Use insect repellent containing DEET or other effective ingredients.  6. Wear light clothing that
covers


Message(message='There are several ways to prevent dengue:\n\n1. Wash your hands frequently with soap and water, especially before eating, after using the bathroom, and after blowing your nose or coughing.\n\n2. Wear a face mask when in close proximity to others, especially during peak mosquito hours (dawn and dusk).\n\n3. Avoid standing under tall trees, near stagnant water, and near standing water.\n\n4. Stay in air-conditioned or air-conditioned rooms.\n\n5. Use insect repellent containing DEET or other effective ingredients.\n\n6. Wear light clothing that covers', sender='Bloom')